**Training a Network with tf.functions**

In [1]:
# importing required libraries and packages, which will be used through code.
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pathlib

In [2]:
# this part downloads the cat vs dog dataset from the google servers, and extracts them. finally, you will have train and validation data paths in two variables.
URL =     'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=URL, extract=True)

PATH =          os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir                    =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir           = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

68616192/68606236 [==============================] - 0s 0us/step


In [3]:
# first, we have defined the augmentation function with some tf.image attributes.
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 150, 150)
    image = tf.image.random_brightness(image, 0.5) 
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down   (image)
    return image

In [4]:
# Determining class names, will be used in the following function
Class_Names=['cats','dogs']

In [5]:
# Here, we have defined the get_label function, which assigns a one-hot coded label to each file-path
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the     class-directory
  return parts[-2] == Class_Names

In [6]:
# a test on above-mentioned function
get_label("/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.0.jpg").numpy()

array([ True, False])

In [7]:
# Next, we'll define a function to load an image, decode it, and change its tensor type to float 32.
def load_img(image_path):
    img = tf.io.read_file(image_path)
    
    # https://stackoverflow.com/questions/44942729/tensorflowvalueerror-images-contains-no-shape
    img = tf.image.decode_image(img, 3, expand_animations=False)
    
    img = tf.cast(img, tf.float32)
    return img

In [8]:
# A function to normalize the images between [-1, 1]
def normalize(image):
    image =  (image / 127.5)-1
    return image

In [9]:
# Here, the resizing function have been defined, which resizes to an arbitrary size given as input.
def resize(image,height, width):
    image = tf.image.resize(image, (height, width),
                                 method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    #image = tf.image.resize_with_crop_or_pad(image, height, width)
    return image

In [10]:
# a function, which returns image and its label, using load_img and get_label functions,
# corresponding to the given image_path.
def load_image_with_label(image_path):
    label = get_label(image_path)
    img   =  load_img(image_path)
    return img, label

In [11]:
# a Function to load train data, which includes load_image_with_label, augment_data, and normalize functions
def load_image_train(image_file):
    image, label = load_image_with_label(image_file)
    image = augment_data(image)
    image =    normalize(image)
    
    return image, label

In [12]:
# a Function to load test data, which is similar to the previous one, but obviously doesn't include augment_data
def load_image_test(image_file):
    image, label = load_image_with_label(image_file)
    image = resize(image, 150, 150)
    image = normalize(image)

    return image, label

In [13]:
# Determining batch-size and shuffle buffer size, which will be used further.
BATCH_SIZE            = 32
SHUFFLE_BUFFER_SIZE = 1000

In [14]:
# extracting train and validation directories from corresponding paths.
train_dir      =             pathlib.Path(train_dir)
validation_dir =        pathlib.Path(validation_dir)

In [15]:
# training and validation datasets are stored in two ShuffleDataset typed files.
train_dataset     = tf.data.Dataset.list_files     (str(train_dir/'*/*'))
validation_dataset= tf.data.Dataset.list_files(str(validation_dir/'*/*'))

In [16]:
print(len(list(train_dataset))     )
print(len(list(validation_dataset)))

2000
1000


In [17]:
# we will use AUTOTUNE in order to employ prefetch attribute for accelleration.
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [18]:
# Get train data set
train_dataset = train_dataset.map(load_image_train)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE)
train_dataset = train_dataset.    batch(BATCH_SIZE)
train_dataset = train_dataset.   prefetch(AUTOTUNE)

In [19]:
# Get validation data set
validation_dataset = validation_dataset.map(load_image_test)
validation_dataset = validation_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.cache()

In [20]:
# Determinig and compiling a sequential network model 
from keras import layers
from keras import models
from tensorflow.keras import optimizers

base_model = tf.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
base_model.trainable =                                False

flatten_layer = layers.GlobalAveragePooling2D()
dense_layer   = layers.Dense(100, activation='relu')
dropout_layer = layers.Dropout(0.5)
predict_layer = layers.Dense(2, activation='softmax')
model = tf.keras.Sequential([
  base_model   ,
  flatten_layer,
  dense_layer  ,
  dropout_layer,
  predict_layer
])

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

58900480/58889256 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 100)               51300     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 14,766,190
Trainable params: 51,502
Non-trainable params: 14,714,688
_______________________________________

In [22]:
# Finally, it's training time
history = model.fit(train_dataset,
                    epochs=100,
                    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 28s 177ms/step - loss: 0.7052 - acc: 0.5540 - val_loss: 0.6871 - val_acc: 0.5520
Epoch 2/100
63/63 [==============================] - 13s 146ms/step - loss: 0.6662 - acc: 0.6365 - val_loss: 0.6581 - val_acc: 0.6090
Epoch 3/100
63/63 [==============================] - 13s 149ms/step - loss: 0.6357 - acc: 0.6610 - val_loss: 0.6371 - val_acc: 0.6270
Epoch 4/100
63/63 [==============================] - 13s 149ms/step - loss: 0.6086 - acc: 0.6980 - val_loss: 0.6118 - val_acc: 0.6620
Epoch 5/100
63/63 [==============================] - 13s 152ms/step - loss: 0.5864 - acc: 0.7235 - val_loss: 0.5872 - val_acc: 0.7010
Epoch 6/100
63/63 [==============================] - 13s 151ms/step - loss: 0.5644 - acc: 0.7525 - val_loss: 0.5756 - val_acc: 0.6860
Epoch 7/100
63/63 [==============================] - 13s 152ms/step - loss: 0.5474 - acc: 0.7550 - val_loss: 0.5577 - val_acc: 0.7000
Epoch 8/100
63/63 [==============================] - 13s 154ms